In [90]:
# --- START OF FILE test.py ---

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
# %matplotlib inline
import datetime
import ccxt

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline,convert_daily_return_to_pyfolio_ts
from finrl.meta.data_processor import DataProcessor
import sys
sys.path.append("../FinRL-Library")
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

class CCXTDownloader:
    def __init__(self, exchange_name='binance', start_date='2020-01-01', end_date='2023-01-01', timeframe='1d'):
        self.exchange = getattr(ccxt, exchange_name)()
        self.start_date = start_date
        self.end_date = end_date
        self.timeframe = timeframe

    def get_data(self, ticker_list):
        all_data = []
        for ticker in ticker_list:
            print(f"Downloading {ticker} from {self.exchange.id}...")
            symbol = ticker + '/USDT' # Assuming USDT pairs for crypto
            
            # Convert start_date to milliseconds timestamp
            since = self.exchange.parse8601(self.start_date + 'T00:00:00Z')
            
            ohlcv = []
            while since < self.exchange.parse8601(self.end_date + 'T00:00:00Z'):
                fetched_data = self.exchange.fetch_ohlcv(symbol, self.timeframe, since)
                if not fetched_data:
                    break
                ohlcv.extend(fetched_data)
                since = ohlcv[-1][0] + self.exchange.parse_timeframe(self.timeframe) * 1000 # Move to the next timeframe
            
            if ohlcv:
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                df['tic'] = ticker
                all_data.append(df)
            else:
                print(f"No data downloaded for {ticker}")

        if all_data:
            full_df = pd.concat(all_data, axis=0)
            full_df = full_df.sort_values(by=['timestamp', 'tic']).reset_index(drop=True)
            
            # Format to match the required output
            full_df.rename(columns={'timestamp': 'date'}, inplace=True)
            full_df['adjcp'] = full_df['close'] # For crypto, adjcp can be the same as close
            full_df['day'] = full_df['date'].dt.dayofweek
            
            return full_df[['date', 'open', 'high', 'low', 'close', 'adjcp', 'volume', 'tic', 'day']]
        else:
            return pd.DataFrame()

downloader = CCXTDownloader(start_date=config.TRAIN_START_DATE, end_date=config.TRAIN_END_DATE)
df = downloader.get_data(config_tickers.CRYPTO_10_TICKER)
print(f"DataFrame shape after data download: {df.shape}")

fe = FeatureEngineer(
                    use_technical_indicator=True,
                    use_turbulence=False,
                    user_defined_feature = False)

df = fe.preprocess_data(df)
print(f"DataFrame shape after preprocessing: {df.shape}")

# --- CORRECTED DATA PROCESSING PIPELINE ---

# 1. Split data FIRST
train = data_split(df, '2018-04-30','2020-07-01')

# --- NEW STEP TO DENSIFY THE TRAINING DATA ---
# This ensures that every date has a row for every ticker, preventing shape mismatches.
print("Densifying training data...")
# Get the full range of dates and all unique tickers from the training set
all_dates = pd.to_datetime(train['date'].unique())
all_tickers = train['tic'].unique()
multi_index = pd.MultiIndex.from_product([all_dates, all_tickers], names=['date', 'tic'])

# Set the index and reindex to create a complete, dense frame.
# This will introduce NaNs for dates/tickers that were missing.
train = train.set_index(['date', 'tic'])
train = train.reindex(multi_index)

# Group by ticker and forward-fill the missing data.
# bfill is used to fill any remaining NaNs at the beginning of the data.
train = train.groupby(level='tic').ffill().bfill()

# Reset the index to bring 'date' and 'tic' back as columns
train.reset_index(inplace=True)
print("Data densification complete.")

# 2. Function to calculate and add covariance matrix to a dataframe
def add_covariance_matrix(data):
    # Sort and factorize index
    data = data.sort_values(['date', 'tic'], ignore_index=True)
    data.index = data.date.factorize()[0]
    
    cov_list = []
    return_list = []
    # look back is one year
    lookback = 252
    
    for i in range(lookback, len(data.index.unique())):
        data_lookback = data.loc[i - lookback:i, :]
        price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
        return_lookback = price_lookback.pct_change().dropna()
        return_list.append(return_lookback)

        if not return_lookback.empty and len(return_lookback) > 1:
            covs = return_lookback.cov().values
        else:
            # Handle case with insufficient data
            num_tickers = len(data['tic'].unique())
            covs = np.zeros((num_tickers, num_tickers))
        cov_list.append(covs)

    # Merge covariance data
    # Make sure to only use dates that have a calculated covariance matrix
    df_cov = pd.DataFrame({
        'date': data.date.unique()[lookback:],
        'cov_list': cov_list,
        'return_list': return_list
    })
    data = data.merge(df_cov, on='date')
    data = data.sort_values(['date', 'tic']).reset_index(drop=True)
    return data

# 3. Apply the function to the train set
train = add_covariance_matrix(train)

# --- FIX: SET A DAY-BASED INDEX FOR THE ENVIRONMENT ---
train.index = train.date.factorize()[0]

# --- END OF CORRECTION ---

import numpy as np
import pandas as pd
from gymnasium.utils import seeding
from gymnasium import spaces
import gymnasium
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import DummyVecEnv


class StockPortfolioEnv(gymnasium.Env):
    """A single stock trading environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step


    """
    metadata = {'render.modes': ['human']}

# --- CORRECTED __init__ METHOD for StockPortfolioEnv ---

    def __init__(self,
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list
        # --- FIX: ADD THE MISSING ATTRIBUTE ---
        self.model_name = self.__class__.__name__

        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))
        
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].iloc[0]
        
        tech_indicator_data = self.data[self.tech_indicator_list].values.T
        self.state = np.vstack((self.covs, tech_indicator_data))

        self.terminal = False
        self.turbulence_threshold = turbulence_threshold
        self.portfolio_value = self.initial_amount

        self.asset_memory = [self.initial_amount]
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()

            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")

            return self.state, self.reward, self.terminal, False, {}

        else:
            weights = self.softmax_normalization(actions)
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].iloc[0]
            
            tech_indicator_data = self.data[self.tech_indicator_list].values.T
            self.state = np.vstack((self.covs, tech_indicator_data))

            # calcualte portfolio return
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])
            self.asset_memory.append(new_portfolio_value)

            self.reward = new_portfolio_value

        return self.state, self.reward, self.terminal, False, {}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        
        self.covs = self.data['cov_list'].iloc[0]
        
        tech_indicator_data = self.data[self.tech_indicator_list].values.T
        self.state = np.vstack((self.covs, tech_indicator_data))

        self.portfolio_value = self.initial_amount
        self.terminal = False
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]
        return self.state, {}

    def render(self, mode='human'):
        return self.state

    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output


    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']

        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        # The .reset() method of a VecEnv returns the initial observation
        obs = e.reset()
        # Return the environment and the initial observation
        return e, obs
        
stock_dimension = len(train.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.001,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4

}

e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

DataFrame shape after data download: (9000, 9)
Successfully added technical indicators
DataFrame shape after preprocessing: (9000, 17)
Densifying training data...
Data densification complete.
Stock Dimension: 8, State Space: 8
<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [91]:
# initialize
agent = DRLAgent(env = env_train)

### Model 1: **A2C**

In [92]:
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [93]:
trained_a2c = agent.train_model(model=model_a2c,
                                tb_log_name='a2c',
                                total_timesteps=50000)

-----------------------------------------
| time/                 |               |
|    fps                | 744           |
|    iterations         | 100           |
|    time_elapsed       | 0             |
|    total_timesteps    | 500           |
| train/                |               |
|    entropy_loss       | -11.3         |
|    explained_variance | 0             |
|    learning_rate      | 0.0002        |
|    n_updates          | 99            |
|    policy_loss        | 8.66e+10      |
|    reward             | 2.5874964e+09 |
|    reward_max         | 2.7905285e+09 |
|    reward_mean        | 2.7006692e+09 |
|    reward_min         | 2.5874964e+09 |
|    std                | 0.996         |
|    value_loss         | 7.22e+19      |
-----------------------------------------
begin_total_asset:1000000
end_total_asset:2892393204.2809153
Sharpe:  0.686938027219528
-----------------------------------------
| time/                 |               |
|    fps                | 653 

In [94]:
trained_a2c.save('./content/trained_models/trained_a2c.zip')

### Model 2: **PPO**

In [68]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [67]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=80000)

begin_total_asset:1000000
end_total_asset:4564164797.908198
Sharpe:  0.6836593088373951
begin_total_asset:1000000
end_total_asset:3399527914.014721
Sharpe:  0.6854340164296723
begin_total_asset:1000000
end_total_asset:4042943901.547427
Sharpe:  0.6891093076984665
--------------------------------------
| time/              |               |
|    fps             | 970           |
|    iterations      | 1             |
|    time_elapsed    | 2             |
|    total_timesteps | 2048          |
| train/             |               |
|    reward          | 2.9896799e+09 |
|    reward_max      | 6.231379e+09  |
|    reward_mean     | 1.2438243e+09 |
|    reward_min      | 847152.3      |
--------------------------------------
begin_total_asset:1000000
end_total_asset:2667665258.968956
Sharpe:  0.6862526864109054
begin_total_asset:1000000
end_total_asset:2042807941.9065382
Sharpe:  0.6860951858379205
begin_total_asset:1000000
end_total_asset:8398144979.394912
Sharpe:  0.6845743041959327
beg

In [70]:
trained_ppo.save('./content/trained_models/trained_ppo.zip')

### Model 3: **DDPG**


In [ ]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cuda device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4981447.8010044675
Sharpe:  0.8613296544165371
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 174       |
|    time_elapsed    | 66        |
|    total_timesteps | 11572     |
| train/             |           |
|    actor_loss      | -8.42e+07 |
|    critic_loss     | 5.81e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 8679      |
|    reward          | 5267437.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asset:1000000
end_total_asset:5267436.861472457
Sharpe:  0.8789112048343828
begin_total_asse

In [ ]:
trained_ddpg.save('./content/trained_models/trained_ddpg.zip')

### Model 4: **SAC**


In [71]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [72]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000)

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

In [73]:
trained_sac.save('./content/trained_models/trained_sac.zip')

### Model 5: **TD3**


In [74]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [75]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=30000)

Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging Error: 'rollout_buffer'
Logging 

In [76]:
trained_td3.save('./content/trained_models/trained_td3.zip')

## Trading
Assume that we have $1,000,000 initial capital at 2019-01-01. We use the A2C model to trade Dow jones 30 stocks.

In [95]:
# Apply the same processing to the trade dataframe
trade = data_split(df, '2020-07-01', '2021-10-31')

# --- NEW STEP: Ensure consistent tickers across train and trade sets ---
# Filter the trade data to only include tickers present in the training data.
# This is crucial for ensuring the observation and action spaces match the trained model.
train_tickers = train.tic.unique()
trade = trade[trade['tic'].isin(train_tickers)].reset_index(drop=True)
print(f"Tickers in train: {len(train_tickers)}. Tickers in trade after filtering: {len(trade.tic.unique())}")


# 1. Densify the trading data
print("Densifying trading data...")
all_dates_trade = pd.to_datetime(trade['date'].unique())
all_tickers_trade = trade['tic'].unique()
multi_index_trade = pd.MultiIndex.from_product([all_dates_trade, all_tickers_trade], names=['date', 'tic'])

trade = trade.set_index(['date', 'tic'])
trade = trade.reindex(multi_index_trade)
trade = trade.groupby(level='tic').ffill().bfill()
trade.reset_index(inplace=True)
print("Trading data densification complete.")

# 2. Add covariance matrix to the trade set
trade = add_covariance_matrix(trade)

# 3. Set the day-based index for the environment
trade.index = trade.date.factorize()[0]

# --- END OF FIX ---

# Now, the trade environment will have the same dimensions as the train environment
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)


Tickers in train: 8. Tickers in trade after filtering: 8
Densifying trading data...
Trading data densification complete.


In [96]:
trade.shape

(1728, 19)

In [97]:
df_daily_return, df_actions = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:1000000
end_total_asset:1134432.9448877855
Sharpe:  0.5803414343254688
hit end!


In [98]:
df_daily_return.head()

,date,daily_return
0,2021-03-10,0.000000
1,2021-03-11,0.010866
2,2021-03-12,-0.014560
3,2021-03-13,0.040658
4,2021-03-14,-0.022750


In [ ]:
df_daily_return.to_csv('df_daily_return.csv')

In [99]:
df_actions.head()

,ADA,BCH,BTC,ETH,LINK,LTC,MATIC,XRP
date,,,,,,,,
2021-03-10,0.125000,0.125000,0.12500,0.125000,0.125000,0.125000,0.12500,0.125000
2021-03-11,0.074179,0.074179,0.20164,0.159666,0.100483,0.114034,0.20164,0.074179
2021-03-12,0.074179,0.074179,0.20164,0.159666,0.100483,0.114034,0.20164,0.074179
2021-03-13,0.074179,0.074179,0.20164,0.159666,0.100483,0.114034,0.20164,0.074179
2021-03-14,0.074179,0.074179,0.20164,0.159666,0.100483,0.114034,0.20164,0.074179


In [100]:
df_actions.to_csv('df_actions.csv')